# Проверка корректности и эффективности фильтрации дублирующих записей

In [1]:
import warnings

import pandas as pd
import pymssql
import plotly.graph_objs as go
import plotly.io as pio

pd.set_option('display.max_colwidth', None)
pio.renderers.default = "notebook_connected"
warnings.filterwarnings("ignore", category=UserWarning)

In [2]:
def ms_sql_con():
    sql_name = 'voice_ai'
    sql_server = '10.2.4.124'
    sql_login = 'ICECORP\\1c_sql'

    with open('sql.pass','r') as file:
        sql_pass = file.read().replace('\n', '')
        file.close()

    return pymssql.connect(
            server = sql_server,
            user = sql_login,
            password = sql_pass,
            database = sql_name,
            tds_version=r'7.0'
    )

In [3]:
def read_sql(query):
    return pd.read_sql_query(query, con=ms_sql_con(), parse_dates=None)

In [4]:
query_old = '''
SELECT *
FROM (SELECT DISTINCT
             linkedid call_id
      FROM calls
      WHERE CAST(call_date AS DATE) BETWEEN '2024-02-08' AND '2024-02-27'
      AND linkedid IS NOT NULL AND linkedid <> '') c
LEFT JOIN (SELECT DISTINCT
                  linkedid transcribation_id
           FROM transcribations
           WHERE CAST(transcribation_date AS DATE) BETWEEN '2024-02-08' AND '2024-02-27') t
ON c.call_id = t.transcribation_id;
'''

query_new = '''
SELECT *
FROM (SELECT DISTINCT
             linkedid call_id
      FROM calls
      WHERE CAST(call_date AS DATE) BETWEEN '2024-02-28' AND '2024-03-18'
      AND linkedid IS NOT NULL AND linkedid <> '') c
LEFT JOIN (SELECT DISTINCT
                  linkedid transcribation_id
           FROM transcribations
           WHERE CAST(transcribation_date AS DATE) BETWEEN '2024-02-28' AND '2024-03-18') t
ON c.call_id = t.transcribation_id;
'''

In [5]:
old = read_sql(query_old)
new = read_sql(query_new)

In [6]:
print(f'Процент транскрипций в разговорах до фильтрации на дубли - {len(old[old.transcribation_id.notna()]) / len(old) * 100:.2f}%'
)

Процент транскрипций в разговорах до фильтрации на дубли - 99.52%


In [7]:
print(f'Процент транскрипций в разговорах после фильтрации на дубли - {len(new[new.transcribation_id.notna()]) / len(new) * 100:.2f}%'
)

Процент транскрипций в разговорах после фильтрации на дубли - 99.49%


In [8]:
query_old_queue = '''
SELECT AVG(time)
FROM perf_log
WHERE CAST(event_date AS DATE) BETWEEN '2024-02-08' AND '2024-02-27';
'''

query_new_queue = '''
SELECT AVG(time)
FROM perf_log
WHERE CAST(event_date AS DATE) BETWEEN '2024-02-28' AND '2024-03-18';
'''

In [9]:
old_queue = read_sql(query_old_queue)
new_queue = read_sql(query_new_queue)

In [10]:
print(f'Средняя длительность очереди до фильтрации на дубли - {old_queue.values[0][0]:.2f} сек.')

Средняя длительность очереди до фильтрации на дубли - 33.91 сек.


In [11]:
print(f'Средняя длительность очереди после фильтрации на дубли - {new_queue.values[0][0]:.2f} сек.')

Средняя длительность очереди после фильтрации на дубли - 33.24 сек.
